### Import modules

In [230]:
import pandas as pd
import io
import os,sys
import urllib.request, urllib.parse, urllib.error
import pandas_datareader.data as pdr
import datetime
import numpy as np
from tqdm import tqdm

### Create a python function to access CSV data from Intrinio, and put it into a pandas DataFrame

In [68]:
def get_from_intrinio(stk,start_date=None,end_date=None):
    sd = '2016-01-01' if start_date is None else start_date
    ed = str(datetime.datetime.now())[0:10]
    try:
        url = f'https://api.intrinio.com/prices.csv?start_date={sd}&end_date={ed}&identifier={stk}&api_key=OmU1NmEzYTU1Yjg3N2NjYWRiOWFjM2Y2OGYzMWNlMDcx'
        req = urllib.request.Request(url)
        f = urllib.request.urlopen(req)
    except Exception:
        f = urllib.request.urlopen(req)
    alines = f.read().decode('utf-8')        
    csv = alines.split('\n')[1:]
    handle = io.StringIO('\n'.join(csv))
    df = pd.read_csv(handle)
    return df


### Get Data

In [89]:
sym = 'AAPL'
df1 = get_from_intrinio(sym)

In [90]:
df1.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,EX_DIVIDEND,SPLIT_RATIO,ADJ_OPEN,ADJ_HIGH,ADJ_LOW,ADJ_CLOSE,ADJ_VOLUME,ADJ_FACTOR
0,2019-06-14,191.545,193.5863,190.300,192.74,18761474.0,0.0,1.0,191.545,193.5863,190.300,192.74,18761474.0,1.0
1,2019-06-13,194.700,196.7900,193.600,194.15,21674625.0,0.0,1.0,194.700,196.7900,193.600,194.15,21674625.0,1.0
2,2019-06-12,193.950,195.9700,193.385,194.19,18253189.0,0.0,1.0,193.950,195.9700,193.385,194.19,18253189.0,1.0
3,2019-06-11,194.860,196.0000,193.600,194.81,26932882.0,0.0,1.0,194.860,196.0000,193.600,194.81,26932882.0,1.0
4,2019-06-10,191.810,195.3700,191.620,192.58,26220851.0,0.0,1.0,191.810,195.3700,191.620,192.58,26220851.0,1.0


In [71]:
df1.tail()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,EX_DIVIDEND,SPLIT_RATIO,ADJ_OPEN,ADJ_HIGH,ADJ_LOW,ADJ_CLOSE,ADJ_VOLUME,ADJ_FACTOR
863,2016-01-08,133.18,133.816,131.3200,131.63,4762706.0,0.0,1.0,115.424820,115.976031,113.812790,114.081462,4762706.0,1.0
864,2016-01-07,133.70,135.020,132.4300,132.86,7025760.0,0.0,1.0,115.875495,117.019517,114.774808,115.147482,7025760.0,1.0
865,2016-01-06,134.38,135.580,133.6200,135.17,4310939.0,0.0,1.0,116.464840,117.504859,115.806161,117.149519,4310939.0,1.0
866,2016-01-05,136.76,136.890,134.8501,135.85,3924793.0,0.0,1.0,118.527545,118.640214,116.872267,117.738864,3924793.0,1.0
867,2016-01-04,135.60,135.970,134.2400,135.95,5229381.0,0.0,1.0,117.522193,117.842866,116.343504,117.825532,5229381.0,1.0


### Show it's done using yahoo

In [72]:
def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    df['Date'] = df.index
    df.Date = df.Date.astype(str).str.slice(0,10)
    df.columns = df.columns.get_level_values(0)
    df.index = list(range(len(df)))
    return df


### Analyis of gap up's and downs

In [236]:
for sym in tqdm(list(df_comp3.Symbol.values)):
    time.sleep(.01)


100%|██████████| 1153/1153 [00:13<00:00, 85.25it/s]

In [238]:
exch = 'nyse'
df_comp = pd.read_csv(f'/users/bperlman1/downloads/{exch}_companylist.csv')
df_comp.columns = [c.strip() for c in df_comp.columns.values]
num_cols = ['MarketCap','LastSale']
for c in num_cols:
    df_comp[c] = df_comp[c].astype(str).str.replace(',','').astype(float)
df_comp2 = df_comp[(df_comp.MarketCap>100000000) & (df_comp.MarketCap<2000000000)]
df_comp2 = df_comp2[df_comp2.LastSale<=30]
print(f'len(df_comp2): {len(df_comp2)}')
df_comp3 = df_comp2[['Symbol']+num_cols].sort_values('MarketCap')
df_comp3.tail(20)
dict_df_smallcap = {}
sym_list = list(df_comp3.Symbol.values)
for sym in tqdm(sym_list):
    print(f'fetching {sym}')
    try:
        df_temp = fetch_history(sym,'2016-01-01','2019-06-14')
        dict_df_smallcap[sym] = df_temp
    except:
        print(f'no data for {sym}')
print(f'total rows: {np.array([len(dict_df_smallcap[k]) for k in dict_df_smallcap.keys()]).sum()}')
dict_large_pchg = {}
for k in dict_df_smallcap:
    dft = dict_df_smallcap[k]
    dft['Symbol'] = k
    dft['prev_close'] = dft.Close.shift(1)
    dft['pchg'] = dft.Close.pct_change()
    dft2 = dft[dft.pchg.abs()>.49]
    if len(dft2)>0:
        dict_large_pchg[k] = dft2
print(f'len of large pchgs: {len(dict_large_pchg)}')
# print(dict_large_pchg[list(dict_large_pchg.keys())[0]])
df_large_chg = dict_large_pchg[list(dict_large_pchg.keys())[0]]
for k in list(dict_large_pchg.keys())[1:]:
    df_large_chg = df_large_chg.append(dict_large_pchg[k])
    df_large_chg.index = list(range(len(df_large_chg)))
df_large_chg = df_large_chg.sort_values(['Symbol','Date']) 
df_large_chg.to_csv(f'/users/bperlman1/downloads/df_large_chg_{exch}.csv',index=False)



  0%|          | 0/994 [00:00<?, ?it/s]

len(df_comp2): 994
fetching BSD



  0%|          | 1/994 [00:01<23:27,  1.42s/it]

fetching SWZ



  0%|          | 2/994 [00:02<23:01,  1.39s/it]

...

fetching GOLF



100%|█████████▉| 993/994 [16:47<00:01,  1.01s/it]

fetching SSW



100%|██████████| 994/994 [16:48<00:00,  1.01s/it]


total rows: 757152
len of large pchgs: 71
